- [HTTPServer 实现](#HTTPServer实现)
  * [HTTPServer class](#HTTPServer-class)
  * [BaseHTTPRequestHandler](#BaseHTTPRequestHandler-class)
  * [SimpleHTTPRequestHandler class](#SimpleHTTPRequestHandler-class)
  * [CGIHTTPRequestHandler class](#CGIHTTPRequestHandler-class)
  * [Summary](#Summary)
  
# HTTPServer 实现

## HTTPServer class

直接看实现：

```Python
class HTTPServer(socketserver.TCPServer):

    allow_reuse_address = 1    # Seems to make sense in testing environment

    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.server_address[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
```
这个类比较简单，继承TCPServer，只是保存了server name以及设置allow_reuse_address(让socket端口可重用)

## BaseHTTPRequestHandler class
在TCPServer实现中我们提到，一个RequestHandler必须依次执行setup(), handle()和finish()。
BaseHTTPRequestHandler继承socketserver.StreamRequestHandler，并重载了handle()函数，看handle()的实现:
```Python
def handle(self):
    """Handle multiple requests if necessary."""
    self.close_connection = True

    self.handle_one_request()
    while not self.close_connection:
        self.handle_one_request()
        
def handle_one_request(self):
    """Handle a single HTTP request.

    You normally don't need to override this method; see the class
    __doc__ string for information on how to handle specific HTTP
    commands such as GET and POST.

    """
    try:
        self.raw_requestline = self.rfile.readline(65537)
        if len(self.raw_requestline) > 65536:
            self.requestline = ''
            self.request_version = ''
            self.command = ''
            self.send_error(HTTPStatus.REQUEST_URI_TOO_LONG)
            return
        if not self.raw_requestline:
            self.close_connection = True
            return
        if not self.parse_request():
            # An error code has been sent, just exit
            return
        mname = 'do_' + self.command
        if not hasattr(self, mname):
            self.send_error(
                HTTPStatus.NOT_IMPLEMENTED,
                "Unsupported method (%r)" % self.command)
            return
        method = getattr(self, mname)
        method()
        self.wfile.flush() #actually send the response if not already done.
    except socket.timeout as e:
        #a read or a write timed out.  Discard this connection
        self.log_error("Request timed out: %r", e)
        self.close_connection = True
        return
```

hanlde()函数根据self.close_connection的值处理一个或多个request。这里如果是"HTTP/1.1"且header设了"keep-alive"，self.close_connection就会被置为false，handle因此可以处理多个request。

handle_one_request()是处理一个request的具体流程，它首先调用parse_request来解析HTTP包，然后根据包里指定的HTTP Method调用对应的函数。比如do_GET, do_POST等。**注意，BaseHTTPRequestHandler里没有实现任何do_XXX的函数。**

## SimpleHTTPRequestHandler class
SimpleHTTPRequestHandler继承BaseHTTPRequestHandler，并且实现了do_GET和do_HEAD方法。
```Python
def do_GET(self):
    """Serve a GET request."""
    f = self.send_head()
    if f:
        try:
            self.copyfile(f, self.wfile)
        finally:
            f.close()

def do_HEAD(self):
    """Serve a HEAD request."""
    f = self.send_head()
    if f:
        f.close()
```

send_head方法发送response code以及MIME headers，并且返回一个文件对象，表示请求获得的资源文件。

因此，do_GET就是将资源写入到socket stream中。而do_HEAD发完header就结束了。

send_head的具体实现也很简单。它首先找到request指定路径下的资源文件(step1)，然后根据资源设置相应的header(step2)，最后返回资源文件。
```Python
def send_head(self):
    # Step 1: Find index.html and index.htm in given path.
    path = self.translate_path(self.path)
    f = None
    if os.path.isdir(path):
        parts = urllib.parse.urlsplit(self.path)
        if not parts.path.endswith('/'):
            # redirect browser - doing basically what apache does
            self.send_response(HTTPStatus.MOVED_PERMANENTLY)
            new_parts = (parts[0], parts[1], parts[2] + '/',
                         parts[3], parts[4])
            new_url = urllib.parse.urlunsplit(new_parts)
            self.send_header("Location", new_url)
            self.end_headers()
            return None
        for index in "index.html", "index.htm":
            index = os.path.join(path, index)
            if os.path.exists(index):
                path = index
                break
        else:
            return self.list_directory(path)
    # Step 2: Set appropriate HTTP code and header.
    ctype = self.guess_type(path)
    try:
        f = open(path, 'rb')
    except OSError:
        self.send_error(HTTPStatus.NOT_FOUND, "File not found")
        return None
    try:
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", ctype)
        fs = os.fstat(f.fileno())
        self.send_header("Content-Length", str(fs[6]))
        self.send_header("Last-Modified", self.date_time_string(fs.st_mtime))
        self.end_headers()
        return f
    except:
        f.close()
        raise
```

## CGIHTTPRequestHandler class

CGIHTTPRequestHandler继承SimpleHTTPRequestHandler，并且加了do_POST方法。 do_POST遵循cgi协议规范，在unix环境中通过fork进程来执行cgi脚本，在non-unix环境中用subprocess.Popen来执行cgi脚本。同时，填入cgi环境变量，传给cgi脚本。 

## Summary
1. HTTPServer就是一个TCPServer.
2. SimpleHTTPRequestHandler实现了简单的HEAD和GET.